In [1]:
import os
import shutil
import sys

# 1. Remove a pasta local 'ltn' se ela existir (para não conflitar com a instalada)
if os.path.exists('ltn'):
    print("Pasta local 'ltn' encontrada. Removendo para usar a biblioteca oficial...")
    shutil.rmtree('ltn')
else:
    print("Nenhuma pasta local 'ltn' encontrada (isso é bom).")

# 2. Instala a biblioteca oficial (garantindo a versão correta)
!pip install git+https://github.com/logictensornetworks/LTNtorch

# 3. Hack para forçar o recarregamento sem reiniciar manual (mas reiniciar manual é melhor)
if 'ltn' in sys.modules:
    del sys.modules['ltn']
if 'ltn.core' in sys.modules:
    del sys.modules['ltn.core']

print("\n>>> PASSO CRÍTICO: Vá no menu acima 'Runtime' (Ambiente de Execução) -> 'Restart Session' (Reiniciar Sessão).")
print(">>> DEPOIS: Rode a segunda célula com o código do classificador.")

Nenhuma pasta local 'ltn' encontrada (isso é bom).
  Cloning https://github.com/logictensornetworks/LTNtorch to /tmp/pip-req-build-0mbvn60q
  Running command git clone --filter=blob:none --quiet https://github.com/logictensornetworks/LTNtorch /tmp/pip-req-build-0mbvn60q
  Resolved https://github.com/logictensornetworks/LTNtorch to commit d1bd98169cc2121f8cdd25ff99901e4589923c95
  Preparing metadata (setup.py) ... done

>>> PASSO CRÍTICO: Vá no menu acima 'Runtime' (Ambiente de Execução) -> 'Restart Session' (Reiniciar Sessão).
>>> DEPOIS: Rode a segunda célula com o código do classificador.


In [4]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import ltn

# --- BLOCO DE SEGURANÇA DE IMPORT (A correção para o erro) ---
# Tenta importar Connective, Quantifier, etc. do ltn.core
try:
    Connective = ltn.core.Connective
    Quantifier = ltn.core.Quantifier
    Predicate = ltn.core.Predicate
    Variable = ltn.core.Variable
except AttributeError:
    # Se falhar (por conta de versões muito antigas ou inesperadas), tenta o caminho direto
    try:
        Connective = ltn.Connective
        Quantifier = ltn.Quantifier
        Predicate = ltn.Predicate
        Variable = ltn.Variable
    except AttributeError:
        print("ERRO DE IMPORT: Não foi possível encontrar as classes. Verifique se o restart foi feito.")
        raise
# --------------------------------------------------------------------

# 1. DADOS (Gatos vs Cachorros sintético)
amount_data = 100
data_class_0 = torch.randn(amount_data, 2)          # Gatos (Centro 0,0)
data_class_1 = torch.randn(amount_data, 2) + 4      # Cachorros (Centro 4,4)

# 2. MODELO (Predicado)
class ClassifierModel(nn.Module):
    def __init__(self): # Corrigido: __init__ com dois underscores
        super(ClassifierModel, self).__init__() # Corrigido: __init__ com dois underscores
        self.elu = nn.ELU()
        self.sigmoid = nn.Sigmoid()
        self.dense1 = nn.Linear(2, 16)
        self.dense2 = nn.Linear(16, 16)
        self.dense3 = nn.Linear(16, 1)

    def forward(self, x):
        x = self.elu(self.dense1(x))
        x = self.elu(self.dense2(x))
        return self.sigmoid(self.dense3(x))

# Definindo os símbolos LTN
P = Predicate(ClassifierModel())
var_0 = Variable("var_0", data_class_0)
var_1 = Variable("var_1", data_class_1)

# Operadores Lógicos
Not = Connective(ltn.fuzzy_ops.NotStandard())
Forall = Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
SatAgg = ltn.fuzzy_ops.SatAgg()

# 3. TREINAMENTO
optimizer = torch.optim.Adam(P.model.parameters(), lr=0.001)
epochs = 1000

print("Iniciando treinamento...")
losses = []
for epoch in range(epochs):
    optimizer.zero_grad()

    # Axiomas:
    # "Para todo x da classe 1, P(x) é verdade"
    axiom1 = Forall(var_1, P(var_1))
    # "Para todo x da classe 0, P(x) NÃO é verdade"
    axiom2 = Forall(var_0, Not(P(var_0)))

    # Satisfação total
    sat = SatAgg(axiom1, axiom2)
    loss = 1. - sat

    loss.backward()
    optimizer.step()

    losses.append(loss.item())
    if epoch % 100 == 0:
        print(f"Epoch {epoch} | Loss: {loss.item():.4f} | Sat: {sat.item():.4f}")

# 4. VISUALIZAÇÃO (Omitido para brevidade, mas o código completo irá rodar)
# ... (código de visualização aqui) ...

Iniciando treinamento...
Epoch 0 | Loss: 0.4524 | Sat: 0.5476
Epoch 100 | Loss: 0.1759 | Sat: 0.8241
Epoch 200 | Loss: 0.0856 | Sat: 0.9144
Epoch 300 | Loss: 0.0751 | Sat: 0.9249
Epoch 400 | Loss: 0.0726 | Sat: 0.9274
Epoch 500 | Loss: 0.0716 | Sat: 0.9284
Epoch 600 | Loss: 0.0711 | Sat: 0.9289
Epoch 700 | Loss: 0.0708 | Sat: 0.9292
Epoch 800 | Loss: 0.0706 | Sat: 0.9294
Epoch 900 | Loss: 0.0705 | Sat: 0.9295
